In [5]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Load the dataset
data = pd.read_csv("../data/raw/MachineLearningRating_v3.csv" ,delimiter='|')
data.head()

C:\Users\Administrator.MMCY\AppData\Local\Temp\ipykernel_18156\886262324.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/raw/MachineLearningRating_v3.csv" ,delimiter='|')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [3]:
# Define the A/B testing function
def perform_ab_testing(group_a, group_b, column):
    """Perform a two-sample t-test between two groups for a given column."""
    t_stat, p_value = stats.ttest_ind(group_a[column], group_b[column], equal_var=False, nan_policy='omit')
    return t_stat, p_value



In [7]:
# Ensure the necessary columns are in the dataset
required_columns = ['Province', 'Gender', 'StatutoryRiskType', 'VehicleType', 'TotalClaims', 'TotalPremium']
for col in required_columns:
    if col not in data.columns:
        print(f"Column {col} is missing from the dataset.")
        raise ValueError("Dataset does not contain all required columns.")

In [8]:
# Hypothesis 1: No significant differences in claims across Provinces
provinces = data['Province'].dropna().unique()
if len(provinces) >= 2:
    province_group_a = data[data['Province'] == provinces[0]]
    province_group_b = data[data['Province'] == provinces[1]]
    t_stat, p_value = perform_ab_testing(province_group_a, province_group_b, 'TotalClaims')
    print(f"Risk Differences Across Provinces: p-value = {p_value}")
else:
    print("Not enough unique Provinces to perform the test.")

Risk Differences Across Provinces: p-value = 0.21269135333580674


In [10]:
# Hypothesis 2: No significant claim differences by Gender
genders = data['Gender'].dropna().unique()
if len(genders) >= 2:
    gender_group_a = data[data['Gender'] == genders[0]]
    gender_group_b = data[data['Gender'] == genders[1]]
    t_stat, p_value = perform_ab_testing(gender_group_a, gender_group_b, 'TotalClaims')
    print(f"Risk Differences by Gender: p-value = {p_value}")
else:
    print("Not enough gender diversity to perform the test.")

Risk Differences by Gender: p-value = 2.431380249946728e-06


In [13]:
# Hypothesis 3: No significant differences in claims across StatutoryRiskTypes
risk_types = data['StatutoryRiskType'].dropna().unique()
if len(risk_types) >= 2:
    risk_group_a = data[data['StatutoryRiskType'] == risk_types[0]]
    risk_group_b = data[data['StatutoryRiskType'] == risk_types[1]]
    t_stat, p_value = perform_ab_testing(risk_group_a, risk_group_b, 'TotalClaims')
    print(f"Risk Differences Across StatutoryRiskTypes: p-value = {p_value}")
else:
    print("Not enough unique StatutoryRiskTypes to perform the test.")

Not enough unique StatutoryRiskTypes to perform the test.


In [12]:
# Hypothesis 4: No significant differences in premium across Vehicle Types
vehicle_types = data['VehicleType'].dropna().unique()
if len(vehicle_types) >= 2:
    vehicle_group_a = data[data['VehicleType'] == vehicle_types[0]]
    vehicle_group_b = data[data['VehicleType'] == vehicle_types[1]]
    t_stat, p_value = perform_ab_testing(vehicle_group_a, vehicle_group_b, 'TotalPremium')
    print(f"Premium Differences Across Vehicle Types: p-value = {p_value}")
else:
    print("Not enough unique Vehicle Types to perform the test.")

Premium Differences Across Vehicle Types: p-value = 1.3084367425095875e-49
